# Revisão Modelo SEIR 

Data: 14/12/2020
Autora: Fernanda Scovinno


- Revisão de parêmtros
- Validação de projeções com dados de ocupação dos estados, coletados por Leonardo Dantas e Jancer Gomes (estagiários Tech&Dados)

In [1]:
import pandas as pd
import yaml
import requests
from pathlib import Path
import numpy as np

%load_ext autoreload
%autoreload 2

## Confirgurações

In [2]:
states_ids = (
    pd.read_csv("http://datasource.coronacidades.org/br/places/ids")[["state_id", "state_num_id"]]
    .drop_duplicates()
    .set_index("state_num_id")
    .to_dict()["state_id"]
)

states_ids

{12: 'AC',
 27: 'AL',
 13: 'AM',
 16: 'AP',
 29: 'BA',
 23: 'CE',
 53: 'DF',
 32: 'ES',
 52: 'GO',
 21: 'MA',
 31: 'MG',
 50: 'MS',
 51: 'MT',
 15: 'PA',
 25: 'PB',
 26: 'PE',
 22: 'PI',
 41: 'PR',
 33: 'RJ',
 24: 'RN',
 11: 'RO',
 14: 'RR',
 43: 'RS',
 42: 'SC',
 28: 'SE',
 35: 'SP',
 17: 'TO'}

In [3]:
url = "https://raw.githubusercontent.com/ImpulsoGov/farolcovid/stable/src/configs/config.yaml"
config = yaml.load(requests.get(url).text, Loader=yaml.FullLoader)

In [4]:
place_id = "state_id"

## Importa dados

### Ocupação

In [5]:
import os

ocupacao_ufs = os.listdir("data/ocupacao_ufs_20201214")
ocupacao_ufs

['CE.csv', '.ipynb_checkpoints', 'PE.csv', 'SE.csv', 'AC.csv']

In [6]:
ocupacao_path = "data/ocupacao_ufs_20201214/"
ocupacao_ufs = dict()

In [7]:
uf = "CE"
ocupacao_ufs[uf] = pd.read_csv(ocupacao_path+uf+".csv", encoding="utf-8", sep=";")
ocupacao_ufs[uf].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35246 entries, 0 to 35245
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   data                      35246 non-null  object
 1   unidadeSaude              35246 non-null  object
 2   naturezaJuridica          34544 non-null  object
 3   tipoLeito                 35246 non-null  object
 4   quantidadeLeitosAtivos    35246 non-null  int64 
 5   quantidadeLeitosOcupados  35246 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.6+ MB


In [8]:
uf = "PE"
ocupacao_ufs[uf] = pd.read_csv(ocupacao_path+uf+".csv", encoding="utf-8", sep=",")
ocupacao_ufs[uf].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dt_referencia      246 non-null    object 
 1   dt_atualizacao     0 non-null      float64
 2   confirmados        246 non-null    int64  
 3   obitos             246 non-null    int64  
 4   tx_obitos          246 non-null    float64
 5   recuperados        246 non-null    int64  
 6   tx_recuperados     246 non-null    float64
 7   isolamento         246 non-null    int64  
 8   tx_isolamento      246 non-null    float64
 9   enfermaria         246 non-null    int64  
 10  tx_enfermaria      246 non-null    float64
 11  uti                246 non-null    int64  
 12  tx_uti             246 non-null    float64
 13  testes_novos       246 non-null    int64  
 14  testes_acumulados  246 non-null    int64  
 15  tx_testes          246 non-null    int64  
 16  leitos_uti         246 non

In [9]:
uf = "SE"
ocupacao_ufs[uf] = pd.read_csv(ocupacao_path+uf+".csv", encoding="utf-8", sep=",")
ocupacao_ufs[uf].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      251 non-null    object 
 1   confirmados     249 non-null    float64
 2   daily_cases     243 non-null    float64
 3   new_deaths      247 non-null    float64
 4   leitos_uti      221 non-null    float64
 5   leitos_clinico  221 non-null    float64
 6   ocup_uti        251 non-null    int64  
 7   ocup_clinico    251 non-null    int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 15.8+ KB


In [10]:
uf = "AC"
ocupacao_ufs[uf] = pd.read_csv(ocupacao_path+uf+".csv", encoding="utf-8", sep=",")
ocupacao_ufs[uf].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36977 entries, 0 to 36976
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   estado             36977 non-null  object
 1   estadoSigla        36977 non-null  object
 2   municipio          36977 non-null  object
 3   cnes               36977 non-null  int64 
 4   nomeCnes           36977 non-null  object
 5   dataNotificacao    36977 non-null  object
 6   dataAtualizacao    36977 non-null  object
 7   ofertaHospCli      36977 non-null  int64 
 8   ofertaHospUti      36977 non-null  int64 
 9   ofertaSRAGCli      36977 non-null  int64 
 10  ofertaSRAGUti      36977 non-null  int64 
 11  ocupHospCli        36977 non-null  int64 
 12  ocupHospUti        36977 non-null  int64 
 13  ocupSRAGCli        36977 non-null  int64 
 14  ocupSRAGUti        36977 non-null  int64 
 15  altas              36977 non-null  int64 
 16  obitos             36977 non-null  int64

### Casos

In [11]:
# df_cases = pd.read_csv("http://datasource.coronacidades.org/br/states/cases/full")
# df_cases.to_csv("data/br_states_cases_full_20201214.csv")

df_cases = pd.read_csv("data/br_states_cases_full_20201214.csv", index_col=0)
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7450 entries, 0 to 7449
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   active_cases             6888 non-null   float64
 1   cases_mavg               6923 non-null   float64
 2   confirmed_cases          7450 non-null   float64
 3   daily_cases              7450 non-null   float64
 4   daily_cases_growth       7450 non-null   object 
 5   daily_cases_mavg         7288 non-null   float64
 6   daily_cases_mavg_100k    7288 non-null   float64
 7   deaths                   7450 non-null   float64
 8   deaths_mavg              6923 non-null   float64
 9   estimated_cases          6888 non-null   float64
 10  expected_mortality       6923 non-null   float64
 11  infectious_period_cases  7450 non-null   float64
 12  last_updated             7450 non-null   object 
 13  new_deaths               7450 non-null   float64
 14  new_deaths_growth       

### Rt

In [12]:
# df_rt = pd.read_csv("http://datasource.coronacidades.org/br/states/rt")
# df_rt["state_id"] = df_rt["state_num_id"].map(states_ids)
# df_rt.columns = df_rt.columns.str.lower()
# df_rt.to_csv("data/br_states_rt_20201214.csv")

df_rt = pd.read_csv("data/br_states_rt_20201214.csv", index_col=0)
df_rt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rt_high_95                7000 non-null   float64
 1   rt_low_95                 7000 non-null   float64
 2   rt_most_likely            7000 non-null   float64
 3   rt_most_likely_growth     7000 non-null   object 
 4   rt_most_likely_mavg       6838 non-null   float64
 5   rt_most_likely_mavg_100k  6838 non-null   float64
 6   last_updated              7000 non-null   object 
 7   state_num_id              7000 non-null   int64  
 8   data_last_refreshed       7000 non-null   object 
 9   state_id                  7000 non-null   object 
dtypes: float64(5), int64(1), object(4)
memory usage: 601.6+ KB


### Oferta de leitos (CNES)

In [13]:
# df_cnes = pd.read_csv("http://datasource.coronacidades.org/br/cities/cnes")
# df_cnes.groupby("state_id")[["number_beds", "number_icu_beds"]].sum().to_csv("data/br_states_cnes_20201214.csv")

df_cnes = pd.read_csv("data/br_states_cnes_20201214.csv")
df_cnes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   state_id         27 non-null     object
 1   number_beds      27 non-null     int64 
 2   number_icu_beds  27 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 776.0+ bytes


## Trata dados

### Ocupação

In [14]:
ocupacao_ufs["AC"]["dataAtualizacao"] = pd.to_datetime(ocupacao_ufs["AC"]["dataAtualizacao"])
ocupacao_ufs["AC"]["dataNotificacao"] = pd.to_datetime(ocupacao_ufs["AC"]["dataNotificacao"].apply(lambda x: x[:10]))

ocupacao_ufs["AC"].tail()

,estado,estadoSigla,municipio,cnes,nomeCnes,dataNotificacao,dataAtualizacao,ofertaHospCli,ofertaHospUti,ofertaSRAGCli,ofertaSRAGUti,ocupHospCli,ocupHospUti,ocupSRAGCli,ocupSRAGUti,altas,obitos,ocupacaoInformada,data_atualizacao
36972,Acre,AC,Rio Branco,2001578,HOSPITAL GERAL DE CLINICAS DE RIO BRANCO,2020-09-11,2020-09-11 19:00:36.893000+00:00,221,18,97,40,122,10,71,24,5,0,True,2020-12-07 00:00:24.173423
36973,Acre,AC,Rio Branco,2001578,HOSPITAL GERAL DE CLINICAS DE RIO BRANCO,2020-09-05,2020-09-05 15:09:43.287000+00:00,221,18,97,40,124,7,74,35,0,0,True,2020-12-07 00:00:24.173423
36974,Acre,AC,Rio Branco,2001578,HOSPITAL GERAL DE CLINICAS DE RIO BRANCO,2020-09-08,2020-09-09 14:38:41.092000+00:00,221,18,97,40,140,10,75,33,0,2,True,2020-12-07 00:00:24.173423
36975,Acre,AC,Rio Branco,2001578,HOSPITAL GERAL DE CLINICAS DE RIO BRANCO,2020-09-06,2020-09-07 14:05:32.647000+00:00,221,18,97,40,126,10,70,10,15,0,True,2020-12-07 00:00:24.173423
36976,Acre,AC,Cruzeiro do Sul,2000172,SESACRE HOSPITAL DE DERMATOLOGIA SANITARIA,2020-09-15,2020-09-15 16:21:49.036000+00:00,36,0,0,0,20,0,0,0,1,0,True,2020-12-07


In [15]:
ocupacao_ufs["AC"][ocupacao_ufs["AC"]["ocupHospCli"] > ocupacao_ufs["AC"]["ofertaHospCli"]]

,estado,estadoSigla,municipio,cnes,nomeCnes,dataNotificacao,dataAtualizacao,ofertaHospCli,ofertaHospUti,ofertaSRAGCli,ofertaSRAGUti,ocupHospCli,ocupHospUti,ocupSRAGCli,ocupSRAGUti,altas,obitos,ocupacaoInformada,data_atualizacao


In [16]:
cols = {
    "estadoSigla": "state_id",
    "municipio": "city_name",
    "dataAtualizacao": "update",
    "dataNotificacao": "date",
    "ofertaHospUti": "leitos_uti",
    "ofertaHospCli": "leitos_clinico",
    "ocupHospCli": "ocup_clinico",
    "ocupHospUti": "ocup_uti",
    "cnes": "cnes"
}

ocupacao_ufs["AC"] = (ocupacao_ufs["AC"]
                      .query("ocupacaoInformada == True")
                      .rename(columns=cols)[list(cols.values())]
                      .sort_values(["cnes", "date", "update"])
                      .drop_duplicates(subset=["cnes", "date"], keep="last")
                      .groupby(["state_id", "date"])
                      .sum()
                      .reset_index()
                      .drop(columns=["cnes", "state_id"])                      
)

ocupacao_ufs["AC"].tail()

,date,leitos_uti,leitos_clinico,ocup_clinico,ocup_uti
173,2020-09-29,18,221,118,9
174,2020-09-30,18,221,113,10
175,2020-10-01,18,221,114,10
176,2020-10-02,18,221,115,10
177,2020-10-03,18,221,109,10


In [17]:
ocupacao_ufs["SE"] = ocupacao_ufs["SE"].rename(columns={"Unnamed: 0": "date"})
ocupacao_ufs["SE"]["date"] = pd.to_datetime(ocupacao_ufs["SE"]["date"], dayfirst=True)

ocupacao_ufs["SE"] = ocupacao_ufs["SE"].sort_values("date").drop(columns=["confirmados", "daily_cases", "new_deaths"])
ocupacao_ufs["SE"].tail()

,date,leitos_uti,leitos_clinico,ocup_uti,ocup_clinico
4,2020-12-03,179.0,238.0,103,136
3,2020-12-04,179.0,228.0,111,134
2,2020-12-05,179.0,228.0,110,135
1,2020-12-06,179.0,228.0,115,135
0,2020-12-07,179.0,228.0,119,134


In [18]:
ocupacao_ufs["PE"]["dt_referencia"] = pd.to_datetime(ocupacao_ufs["PE"]["dt_referencia"])
ocupacao_ufs["PE"].tail()

,dt_referencia,dt_atualizacao,confirmados,obitos,tx_obitos,recuperados,tx_recuperados,isolamento,tx_isolamento,enfermaria,tx_enfermaria,uti,tx_uti,testes_novos,testes_acumulados,tx_testes,leitos_uti,tx_oc_uti,leitos_enf,tc_oc_enf
241,2020-11-30,NaN,184259,9056,0.049,162589,0.882,116,0.005,891,0.041,83,0.0,9491,770957,80308,829,0.85,806,0.69
242,2020-12-01,NaN,185607,9082,0.049,163498,0.881,117,0.005,903,0.041,85,0.0,5314,776271,80862,829,0.86,806,0.70
243,2020-12-02,NaN,187038,9098,0.049,164955,0.882,98,0.004,922,0.042,82,0.0,4406,780677,81321,829,0.85,791,0.69
244,2020-12-03,NaN,188132,9119,0.048,165979,0.882,85,0.004,941,0.043,84,0.0,5904,786581,81936,829,0.86,791,0.69
245,2020-12-04,NaN,189650,9140,0.048,167137,0.881,82,0.004,929,0.041,86,0.0,7911,794492,82760,829,0.86,791,0.67


In [19]:
cols = {
    "dt_referencia": "date",
    "leitos_uti": "leitos_uti",
    "leitos_enf": "leitos_clinico",
#     "uti": "ocup_uti",
#     "enfermaria": "ocup_clinico"
    "tc_oc_enf": "ocup_clinico",
    "tx_oc_uti": "ocup_uti",
}

ocupacao_ufs["PE"] = ocupacao_ufs["PE"].rename(columns=cols)[cols.values()]

ocupacao_ufs["PE"]["ocup_uti"] = (ocupacao_ufs["PE"]["ocup_uti"] * ocupacao_ufs["PE"]["leitos_uti"]).astype(int)
ocupacao_ufs["PE"]["ocup_clinico"] = (ocupacao_ufs["PE"]["ocup_clinico"] * ocupacao_ufs["PE"]["leitos_clinico"]).astype(int)

ocupacao_ufs["PE"] = ocupacao_ufs["PE"].sort_values("date")
ocupacao_ufs["PE"].tail()

,date,leitos_uti,leitos_clinico,ocup_clinico,ocup_uti
241,2020-11-30,829,806,556,704
242,2020-12-01,829,806,564,712
243,2020-12-02,829,791,545,704
244,2020-12-03,829,791,545,712
245,2020-12-04,829,791,529,712


In [20]:
ocupacao_ufs["CE"]["data"] = pd.to_datetime(ocupacao_ufs["CE"]["data"])
ocupacao_ufs["CE"] = ocupacao_ufs["CE"].rename(columns={"data": "date"}).sort_values("date")

ocupacao_ufs["CE"] = ocupacao_ufs["CE"].pivot_table(index="date", columns="tipoLeito", 
                               values=["quantidadeLeitosOcupados", "quantidadeLeitosAtivos"],
                               aggfunc=sum).reset_index()

ocupacao_ufs["CE"].columns = ["date", "leitos_clinico", "leitos_uti", "ocup_clinico", "ocup_uti"]
ocupacao_ufs["CE"].tail()

,date,leitos_clinico,leitos_uti,ocup_clinico,ocup_uti
219,2020-12-04,1286,435,455,267
220,2020-12-05,960,390,376,257
221,2020-12-06,869,348,364,232
222,2020-12-07,1251,465,499,305
223,2020-12-08,508,247,183,155


## Gráficos de ocupação dos estados

In [21]:
import cufflinks as cf
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
cf.go_offline()

In [22]:
for uf in ocupacao_ufs.keys():
    ocupacao_ufs[uf].set_index("date").iplot(title=f"Leitos UTI e Clínico - {uf}")

In [23]:
for uf in ocupacao_ufs.keys():
    ocupacao_ufs[uf]["tx_ocup_clinico"] = ocupacao_ufs[uf]["ocup_clinico"] / ocupacao_ufs[uf]["leitos_clinico"]
    ocupacao_ufs[uf]["tx_ocup_uti"] = ocupacao_ufs[uf]["ocup_uti"] / ocupacao_ufs[uf]["leitos_uti"]
    
    ocupacao_ufs[uf].set_index("date")[["tx_ocup_clinico", "tx_ocup_uti"]].iplot(title=f"Taxa de ocupação UTI e Clínico - {uf}")

## Inicialização

### Parâmetros de entrada

- Hospitalização
- Populacionais

#### Taxas de hospitalizacao e mortalidade ponderada por faixa

In [24]:
# Muda taxa para razao clinico / uti ~ 2
# config["br"]["seir_parameters"]["i2_percentage"] = 0.095
# config["br"]["seir_parameters"]["i3_percentage"] = 0.05

In [25]:
import generate_states_parameters
place_specific_params = generate_states_parameters.now(config)
place_specific_params["state_id"] = place_specific_params["state_num_id"].map(states_ids)

place_specific_params = place_specific_params.set_index(place_id).loc[ocupacao_ufs.keys()]
place_specific_params

,state_num_id,hospitalized_by_age_perc,i2_percentage,i3_percentage,i1_percentage,fatality_ratio
state_id,,,,,,
CE,23,0.039715,0.032867,0.006847,0.960285,0.006366
PE,26,0.039838,0.032970,0.006869,0.960162,0.006286
SE,28,0.037297,0.030867,0.006431,0.962703,0.005435
AC,12,0.029194,0.024160,0.005033,0.970806,0.003882


#### Parâmetros epidemiológicos e recursos

- Casos
- Mortes
- Rt
- Leitos UTI
- Leitos clínicos

In [26]:
df_cases["last_updated"] = pd.to_datetime(df_cases["last_updated"])
df_rt["last_updated"] = pd.to_datetime(df_rt["last_updated"])

In [27]:
import datetime

pop_params_ufs = pd.DataFrame()

for uf in ocupacao_ufs.keys():
    
    start_date = ocupacao_ufs[uf]["date"].max() - datetime.timedelta(15)
    ocupacao_ufs[uf] = ocupacao_ufs[uf][ocupacao_ufs[uf]["date"] >= start_date]
    
    cases = df_cases[(df_cases["state_id"] == uf) & (df_cases["last_updated"] == start_date)]
    
    rt_date = start_date - datetime.timedelta(10)
    rt = df_rt[(df_rt["state_id"] == uf) & (df_rt["last_updated"] == rt_date)]
    
    cnes = df_cnes[df_cnes[place_id] == uf]
    
    pop_params_ufs = pop_params_ufs.append(
        cases
        .set_index(place_id)
        .join(rt.set_index(place_id)[["rt_most_likely", "rt_low_95", "rt_high_95"]])
        .join(cnes.set_index(place_id))
    )
    
pop_params_ufs

,active_cases,cases_mavg,confirmed_cases,daily_cases,daily_cases_growth,daily_cases_mavg,daily_cases_mavg_100k,deaths,deaths_mavg,estimated_cases,...,population,state_name,state_num_id,total_estimated_cases,data_last_refreshed,rt_most_likely,rt_low_95,rt_high_95,number_beds,number_icu_beds
state_id,,,,,,,,,,,,,,,,,,,,,
CE,27901.0,993.266667,293249.0,1632.0,estabilizando,939.3,10.285720,9514.0,7.466667,1943.0,...,9132078,Ceará,23,1517535.0,2020-12-14 12:32:51,0.99,0.87,1.09,12685,785
PE,39107.0,665.466667,173624.0,908.0,estabilizando,677.7,7.091085,8890.0,14.866667,2768.0,...,9557071,Pernambuco,26,1438002.0,2020-12-14 12:32:51,0.97,0.84,1.08,15921,1406
SE,10923.0,134.800000,87248.0,159.0,crescendo,171.9,7.478153,2275.0,2.400000,787.0,...,2298696,Sergipe,28,427645.0,2020-12-14 12:32:51,0.89,0.73,1.03,2552,248
AC,4104.0,120.466667,26942.0,226.0,estabilizando,134.0,15.193863,647.0,1.666667,292.0,...,881935,Acre,12,170652.0,2020-12-14 12:32:51,0.97,0.79,1.11,1135,54


## Simulação

In [28]:
import prepare_params
import seir
import simulator

simulations = {k: dict() for k in [0.3, 0.4, 0.5]}

for prop in simulations.keys():
    
    # Cria tabela de parametros para cada estado
    simulations[prop]["params"] = (pop_params_ufs.apply(lambda row: prepare_params.main(row, place_id, 
                                                                           config, place_specific_params, 
                                                                           symptomatic=prop), axis=1))
    
    # Cria tabela de SEIR inicial para cada estado (somente para verificar, é tambem gerada dentro da simulacao)
    t = simulations[prop]["params"].apply(lambda row: 
            seir.prepare_states(row["population_params"], 
                                row["place_specific_params"], 
                                config["br"]["seir_parameters"]))
    
    
    simulations[prop]["initial_states"] = pd.DataFrame([i for i in t], index=simulations[prop]["params"].index)

    
    # Roda simulacao
    simulations[prop]["result"] = dict()
    simulations[prop]["compare"] = dict()

    for uf in simulations[prop]["params"].keys():
        # Resultado de 90 dias
        simulations[prop]["result"][uf] = simulator.run_simulation(simulations[prop]["params"][uf], config)
        # Comparacao 15 dias
        simulations[prop]["compare"][uf] = pd.concat([
            ocupacao_ufs[uf].reset_index(),
            simulations[prop]["result"][uf]["best"][["I2", "I3"]].join(simulations[prop]["result"][uf]["worst"][["I2", "I3"]], 
                                                  lsuffix="_best", 
                                                  rsuffix="_worst")
        ], axis=1).loc[:15]

In [29]:
simulations[prop]["initial_states"]

,S,E,I1,I2,I3,R,D
state_id,,,,,,,
CE,8.548386e+06,10046.986393,13395.981857,458.497773,95.520369,550181,9514
PE,8.820804e+06,14080.528440,18774.037920,644.658273,134.303807,693743,8890
SE,1.785375e+06,3942.989439,5257.319252,168.563378,35.117370,501642,2275
AC,8.151369e+05,1494.071106,1992.094808,49.576711,10.328481,62605,647


In [30]:
ocupacao_inicial = pd.DataFrame()
for uf in ocupacao_ufs.keys():
    ocupacao_inicial[uf] = ocupacao_ufs[uf][["ocup_clinico","ocup_uti"]].iloc[0]
    
ocupacao_inicial = ocupacao_inicial.T

In [31]:
ocupacao_inicial.join(simulations[prop]["initial_states"][["I2", "I3"]].astype(int))

,ocup_clinico,ocup_uti,I2,I3
CE,436,259,458,95
PE,476,634,644,134
SE,99,63,168,35
AC,111,9,49,10


## Resultados por UF

In [32]:
def plot_compare(uf, tipo_leito="Clínico"):
    
    cols = {
        "Clínico": ["ocup_clinico", "I2_best", "I2_worst"],
        "UTI": ["ocup_uti", "I3_best", "I3_worst"],
    }
    
    for prop in simulations.keys():
        (simulations[prop]["compare"][uf]
         .set_index("date")[cols[tipo_leito]]
         .iplot(title=f"Ocupação {tipo_leito} real x estimado - {uf}; sintomáticos: {prop}"))

In [33]:
ocupacao_ufs.keys()

dict_keys(['CE', 'PE', 'SE', 'AC'])

### Leitos Clínicos

In [34]:
tipo_leito = "Clínico"

In [35]:
uf = "CE"
plot_compare(uf, tipo_leito)

In [36]:
uf = "PE"
plot_compare(uf, tipo_leito)

In [37]:
uf = "SE"
plot_compare(uf, tipo_leito)

In [38]:
uf = "AC"
plot_compare(uf, tipo_leito)

### Leitos UTI - resultado por UF

In [39]:
tipo_leito = "UTI"

In [40]:
uf = "CE"
plot_compare(uf, tipo_leito)

In [41]:
uf = "PE"
plot_compare(uf, tipo_leito)

In [42]:
uf = "SE"
plot_compare(uf, tipo_leito)

In [43]:
uf = "AC"
plot_compare(uf, tipo_leito)

## Análises

- CE, SE e PE estão ainda na faixa nos ultimos 15 dias para leitos Clínicos para sintomáticos ~40% (dados de nov/dez)
- Por outro lado, AC muito sobrehospitalizado em leitos clínicos (dados de set/out)
- CE, SE e PE estão **muito sobrehospitalizados para leitos UTI nos ultimos 15 dias** - enquanto AC se encontra na faixa ~40%


### Razao entre pacientes clinico e UTI nos estados

In [44]:
razao_cli_uti = dict()

for uf in ocupacao_ufs.keys():
    razao_cli_uti[uf] = (ocupacao_ufs[uf]["ocup_clinico"] / ocupacao_ufs[uf]["ocup_uti"]).values

razao_cli_uti = pd.DataFrame(razao_cli_uti)
razao_cli_uti

,CE,PE,SE,AC
0,1.683398,0.750789,1.571429,12.333333
1,1.787879,0.734446,1.421875,12.700000
2,1.727626,0.743472,1.279412,10.800000
3,1.704545,0.781874,1.285714,13.200000
4,1.625430,0.794163,1.238095,15.777778
5,1.637500,0.792945,1.452381,11.600000
6,1.514019,0.814417,1.506024,11.500000
7,1.673913,0.842424,1.322581,9.800000
8,1.586572,0.842424,1.285714,10.400000
9,1.602041,0.784884,1.322917,6.700000


In [45]:
proj_cli_uti = {"best": dict(), "worst": dict()}
prop = 0.4

for bound in proj_cli_uti.keys():
    for uf in ocupacao_ufs.keys():
        proj_cli_uti[bound][uf] = (
            simulations[prop]["result"][uf][bound]["I2"] / simulations[prop]["result"][uf][bound]["I3"]
        ).values[:15]

    proj_cli_uti[bound] = pd.DataFrame(proj_cli_uti[bound])

In [46]:
razao_cli_uti = (proj_cli_uti["worst"]
 .join(proj_cli_uti["best"], lsuffix="_worst", rsuffix="_best")
 .join(razao_cli_uti))

razao_cli_uti = razao_cli_uti.reindex(sorted(razao_cli_uti.columns), axis=1)

In [47]:
razao_cli_uti

,AC,AC_best,AC_worst,CE,CE_best,CE_worst,PE,PE_best,PE_worst,SE,SE_best,SE_worst
0,12.333333,4.800000,4.800000,1.683398,4.800000,4.800000,0.750789,4.800000,4.800000,1.571429,4.800000,4.800000
1,12.700000,4.852999,4.853551,1.787879,4.844596,4.844992,0.734446,4.844363,4.844792,1.421875,4.845560,4.846020
2,10.800000,4.868582,4.872264,1.727626,4.855166,4.857805,0.743472,4.854113,4.856974,1.279412,4.851808,4.854873
3,13.200000,4.858262,4.868647,1.704545,4.842784,4.850235,0.781874,4.840076,4.848151,1.285714,4.828196,4.836823
4,15.777778,4.830656,4.851306,1.625430,4.815515,4.830344,0.794163,4.810311,4.826375,1.238095,4.782966,4.800077
5,11.600000,4.792078,4.826056,1.637500,4.779117,4.803543,0.792945,4.770688,4.797138,1.452381,4.722898,4.750983
6,11.500000,4.747064,4.796753,1.514019,4.737637,4.773397,0.814417,4.725413,4.764118,1.506024,4.653349,4.694321
7,9.800000,4.698825,4.765914,1.673913,4.693877,4.742207,0.842424,4.677449,4.729740,1.322581,4.578417,4.633619
8,10.400000,4.649594,4.735137,1.586572,4.649740,4.711424,0.842424,4.628847,4.695566,1.285714,4.501156,4.571424
9,6.700000,4.600891,4.705405,1.602041,4.606491,4.681922,0.784884,4.580999,4.662566,1.322917,4.423781,4.509537


### Leitos por estado

In [49]:
ocupacao_ufs[uf][["leitos_uti", "leitos_clinico"]].iloc[-1]

leitos_uti         18
leitos_clinico    221
Name: 177, dtype: int64

In [50]:
leitos = pd.DataFrame()
for uf in ocupacao_ufs.keys():
    leitos[uf] = ocupacao_ufs[uf][["leitos_clinico", "leitos_uti"]].iloc[-1]
    
leitos = leitos.T

leitos.join(df_cnes[df_cnes["state_id"].isin(ocupacao_ufs.keys())].set_index("state_id"))

,leitos_clinico,leitos_uti,number_beds,number_icu_beds
CE,508.0,247.0,12685,785
PE,791.0,829.0,15921,1406
SE,228.0,179.0,2552,248
AC,221.0,18.0,1135,54
